## Моделирование газовой динамики (распад произвольного разрыва)

In [75]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, ArtistAnimation
from tqdm import tqdm_notebook
from numba import jit

In [76]:
@jit
def P(ro, u, e):
    """
    ro, u, e - scalars
    returns the value of P in this point 
    """
    return ro*(e-u**2/2)*(gamma-1)


@jit
def critical_velocities(p1, ro1, p2, ro2, gamma, u1, u2):
    """
    Calculate and print critical velocities, find configuration
    """
    c1 = sqrt(gamma*p1/ro1)
    c2 = sqrt(gamma*p2/ro2)
    delta_u = u1-u2
    
    U_sh = (p2-p1)/sqrt(ro1*((gamma+1)*p2+(gamma+1)*p1)/2)
    U_out = -2*c2/(gamma-1)*(1-(p1/p2)**((gamma-1)/(2*gamma)))
    U_vac = -2*(c1+c2)/(gamma-1)
    
    print("u1 - u2 = ", delta_u)
    print("U_sh = ", U_sh)
    print("U_out = ", U_out)
    print("U_vac = ", U_vac)
    
    if delta_u >= U_sh:
        print("\nвправо и влево распространяется УВ")
    elif delta_u >= U_out:
        print("\nвлево распространяется УВ, вправо - ВР")
    elif delta_u >= U_vac:
        print("\nвправо и влево распространяются ВР")
    else:
        print("\nвправо и влево распространяются ВР, в центре образуется область вакуума")
    

@jit
def nextLayer(ro, u, e):
    """
    Метод крупных частиц
    ro, u, e - numpy arrays at previuos layer
    returns array with new numpy arrays
    """
    rv = ro.copy()
    uv = u.copy()
    ev = e.copy()
    
    new_ro = ro.copy()
    new_u = u.copy()
    new_e = e.copy()
    
    for i in range(1, N-1):
        pp = (P(ro[i], u[i], e[i]) + P(ro[i+1], u[i+1], e[i+1]))/2
        pn = (P(ro[i], u[i], e[i]) + P(ro[i-1], u[i-1], e[i-1]))/2
        
        up = (u[i] + u[i+1])/2
        un = (u[i] + u[i-1])/2
        
        uv[i] -= r*(pp-pn)/ro[i]
        ev[i] -= r*(pp*up-pn*un)/ro[i]
        
    for i in range(1, N-1):
        uvp = (uv[i] + uv[i+1])/2
        uvn = (uv[i] + uv[i-1])/2
        
        Dp = 0 if uvp >= 0 else 1
        Dn = 1 if uvn >= 0 else 0
        
        Yp = ro[i]*uvp if uvp >= 0 else -ro[i+1]*uvp
        Yn = ro[i-1]*uvn if uvn >= 0 else -ro[i]*uvn
        
        new_ro[i] = ro[i] +2*r*((Dn-0.5)*Yn + (Dp-0.5)*Yp)
        new_u[i] = uv[i]*ro[i]/new_ro[i] + r/new_ro[i]*(uv[i]*((Dn-1)*Yn+(Dp-1)*Yp)+uv[i-1]*Dn*Yn+uv[i+1]*Dp*Yp)
        new_e[i] = ev[i]*ro[i]/new_ro[i] + r/new_ro[i]*(ev[i]*((Dn-1)*Yn+(Dp-1)*Yp)+ev[i-1]*Dn*Yn+ev[i+1]*Dp*Yp)
        
    return [new_ro, new_u, new_e]


def F(ro, u, e, p):
    """
    returns F(f)
    """
    F = np.zeros((3, N))
    F[0] = ro*u
    F[1] = ro*u*u+p
    F[2] = u*(ro*e+p)
    
    return F


def smoothing(f):
    """
    f - np.array with shape (3, N)
    """
    new_f = f.copy()
    ro = f[0]
    for i in range(2, N-2):
        Dmm = ro[i-1] - ro[i-2]
        Dm = ro[i] - ro[i-1]
        
        Dp = ro[i+1] - ro[i]
        Dpp = ro[i+2] - ro[i+1]
         
        if (Dmm*Dm <= 0) or (Dm*Dp <= 0):
            Qm = f[:,i] - f[:,i-1]
        else:
            Qm = 0
            
        if (Dm*Dp <= 0) or (Dp*Dpp <= 0):
            Qp = f[:,i+1] - f[:,i]
        else:
            Qp = 0

        new_f[:,i] += q*(Qp - Qm)
        
    return new_f 
            

def nextLayer_MK(ro, u, e):
    """
    Метод Мак-Кормака
    ro, u, e - numpy arrays at previuos layer
    returns array with new numpy arrays
    """
    U = ro*u
    E = ro*e
    p = np.array(list(map(P, ro, u, e)))
    
    f = np.zeros((3, N))
    f[0] = ro
    f[1] = ro*u
    f[2] = ro*e
    
    Fn = F(ro, u, e, p)
    
    f_ = f.copy()
    new_f = f.copy()
    
    for i in range(N-1):
        f_[:,i] =  f[:,i] - r*(Fn[:,i+1] - Fn[:,i])
    
    Fn_ = F(f_[0], f_[1]/ro, f_[2]/ro, p)
    
    for i in range(1, N):
        new_f[:,i] = (f[:,i]+f_[:,i])/2 - r/2 * (Fn_[:,i] - Fn_[:,i-1])
        
    new_f = smoothing(new_f)
        
    return [new_f[0], new_f[1]/ro, new_f[2]/ro]

### Метод Мак-Кормака

In [77]:
dt = 0.0005
dx = 0.01

N = 1000
T = 1000

r = dt/dx
gamma = 1.5

# best practise: [0.05, 0.2]
q = 0.1

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

ro[:N//2] = [1]*(N//2)
ro[N//2:] = [1.2]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [1]*(N//2)

e[:N//2] = [1.5]*(N//2)
e[N//2:] = [2.4]*(N//2)


critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
                    gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 11))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'r', linestyle = '-', label = 'Плотность')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '-', label = 'Скорость')
ln3, = ax[2].plot([], [], color = 'k', linestyle = '-', label = 'Энергия')
ln4, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(0, 2)
    ax[1].set_ylim(-2, 2)
    ax[2].set_ylim(0, 3)
    ax[3].set_ylim(0, 2)
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        ro, u, e = nextLayer_MK(ro, u, e)
        p = list(map(P, ro, u, e))
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    return ln1,


frame_const = 10
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

u1 - u2 =  -1.0
U_sh =  0.25373401896661857
U_out =  -0.32185802536256974
U_vac =  -9.017575241644614

вправо и влево распространяются ВР


### Метод крупных частиц (МКЧ)

In [78]:
dt = 0.0005
dx = 0.01

N = 1000
T = 1000

r = dt/dx
gamma = 1.5

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

ro[:N//2] = [1]*(N//2)
ro[N//2:] = [1.2]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [1]*(N//2)

e[:N//2] = [1.5]*(N//2)
e[N//2:] = [2.4]*(N//2)

critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
                    gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 11))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'r', linestyle = '-', label = 'Плотность')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '-', label = 'Скорость')
ln3, = ax[2].plot([], [], color = 'k', linestyle = '-', label = 'Энергия')
ln4, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(0, 2)
    ax[1].set_ylim(-2, 2)
    ax[2].set_ylim(0, 3)
    ax[3].set_ylim(0, 2)
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        ro, u, e = nextLayer(ro, u, e)
        p = list(map(P, ro, u, e))
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    return ln1,


frame_const = 10
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

u1 - u2 =  -1.0
U_sh =  0.25373401896661857
U_out =  -0.32185802536256974
U_vac =  -9.017575241644614

вправо и влево распространяются ВР
